In [6]:
import json
import jsonschema
import random
import itertools
import numpy as np
from jsonschema import validate
from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, ReadoutError, depolarizing_error
from qiskit.circuit.random import random_circuit
from mitiq import zne, ddd
from mitiq.zne.scaling.folding import fold_global, fold_gates_at_random, fold_all
from mitiq.zne.scaling.layer_scaling import get_layer_folding
from mitiq.zne.scaling.identity_insertion import insert_id_layers
from mitiq.benchmarks.ghz_circuits import generate_ghz_circuit

# ZNE:

In [7]:
# Load Schema:

def load(schema_path):
    with open(schema_path, 'r') as file:
        return json.load(file)
    
zne_schema =load('./schema/zne_schema.json')

In [34]:
# Create a test "batch" experiment:

zne_batch_test = {"noise_scaling_factors": [[1, 1.25, 1.5], [1,2,3], [2,4,6]],        # Noise scaling values
    "noise_scaling_method": ["global"],                                               # Folding method
    "extrapolation": ["polynomial", "linear"],                                        # Extrapolation method
    }

## Deconstructing a Batch Dictionary:

In [35]:
# Create single experiments from batch object:

# Define a function to make all combinations of experiments from a "batch dictionary" which should be formatted like the test case above
def make_experiment_list(batch_dict):

    # Initialize empty list where we will append each new experiment
    exp_list = []

    # Make list with all combinations of key values from our "batch dictionary"
    combo_list = list(itertools.product(batch_dict['noise_scaling_factors'], 
                                        batch_dict['noise_scaling_method'], 
                                        batch_dict['extrapolation']))
    # Iterate over the list
    for k in range(len(combo_list)):

        # Initialize single experiment dictionary with keys
        exp = {x: set() for x in ['noise_scaling_factors', 'noise_scaling_method', 'extrapolation']}

        # Map each key to its unique value from our list of combinations
        exp['noise_scaling_factors'] = combo_list[k][0]
        exp['noise_scaling_method'] = combo_list[k][1]
        exp['extrapolation'] = combo_list[k][2]

        # Pull out each experiment
        exp_list.append(exp)

    # Returns a list of experiments
    return exp_list

In [36]:
# Implementing our function using the test case defined above:

make_experiment_list(zne_batch_test)

[{'noise_scaling_factors': [1, 1.25, 1.5],
  'noise_scaling_method': 'global',
  'extrapolation': 'polynomial'},
 {'noise_scaling_factors': [1, 1.25, 1.5],
  'noise_scaling_method': 'global',
  'extrapolation': 'linear'},
 {'noise_scaling_factors': [1, 2, 3],
  'noise_scaling_method': 'global',
  'extrapolation': 'polynomial'},
 {'noise_scaling_factors': [1, 2, 3],
  'noise_scaling_method': 'global',
  'extrapolation': 'linear'},
 {'noise_scaling_factors': [2, 4, 6],
  'noise_scaling_method': 'global',
  'extrapolation': 'polynomial'},
 {'noise_scaling_factors': [2, 4, 6],
  'noise_scaling_method': 'global',
  'extrapolation': 'linear'}]

## Validating a batch of experiments:

In [37]:
# Validate each experiment in our batch object:

# Create function that takes in a batch dictionary and schema to validate against
def batch_validate(batch_dict, schema):

    # Initialize empty list for validation results
    validation_results = []

    # Create list of experiments
    formatted_batch = make_experiment_list(batch_dict)

    # Iterate over list of experiments and individually validate
    for k in formatted_batch:
        try:
            validate(instance=k, schema=schema)
            result = "validation passed"
        except jsonschema.exceptions.ValidationError as e:
            result = "validation failed"

        # Pull out validation results
        validation_results.append(result)
        
    return validation_results

In [38]:
# Implementing our batch validate function using the same test case:

batch_validate(zne_batch_test, zne_schema)

['validation passed',
 'validation passed',
 'validation passed',
 'validation passed',
 'validation passed',
 'validation passed']

## Creating Noise Models:

In [39]:
# Making a thermal noise model using IonQ T1, T2 parameters:

from qiskit_aer.noise import (NoiseModel, thermal_relaxation_error)

def thermal_relaxation_noise_ionq(N):                                     
    
    # T1 and T2 values for qubits 0-N
    T1s = np.random.normal(1.1e10, 0.2e10, N)  # Sampled N values from normal dist, w/ mean = 1.1e10, std = 2e9 us, (converted to ns)
    T2s = np.random.normal(2e8, 0.2e8, N)      # Sampled N values from normal dist, w/ mean = 2e5, std = 2e4 us (converted to ns)

    # Instruction times (in nanoseconds)
    time_ones = 1000   

    noise_thermal = NoiseModel(basis_gates=['h', 'x', 'y', 'z', 'cx'])               # Initialize noise model w ansatz gates

    # QuantumError objects
    errors_ones  = [thermal_relaxation_error(t1, t2, time_ones)            # Make tuples of errors for rx, rz gates 
                for t1, t2 in zip(T1s, T2s)]
    
    # Apply error to all qubits in circuit from randomly sampled parameters
    for j in range(N):
        noise_thermal.add_quantum_error(errors_ones[j], "h", [j])
        noise_thermal.add_quantum_error(errors_ones[j], "x", [j])
        noise_thermal.add_quantum_error(errors_ones[j], "y", [j])
        noise_thermal.add_quantum_error(errors_ones[j], "z", [j])
                
    sim_noise = AerSimulator(noise_model=noise_thermal)
    return sim_noise, noise_thermal

In [40]:
# Ella's basic noise model:

def basic_noise(n_qubits, prob=0.005):

    noise_model = NoiseModel()

    for i in range(n_qubits):
        readout_err = ReadoutError([[0.99, 0.01], [0.2, 0.8]])
        noise_model.add_readout_error(readout_err, [i])
        
    depolarizing_err1 = depolarizing_error(prob, num_qubits=1)
    depolarizing_err2 = depolarizing_error(prob, num_qubits=2)
    noise_model.add_all_qubit_quantum_error(depolarizing_err1, ['h', 'x', 'y', 'z'])
    noise_model.add_all_qubit_quantum_error(depolarizing_err2, ['cx'])

    return noise_model

## Defining the Executor & Batch Executor:

In [41]:
# Single experiment executor (very slightly modified from Ella's):

def execute(circuit, noise_model='basic'):       # Set noise_model to thermal, basic, noiseless
    circ = circuit.copy()
    circ.measure_all()
    num_qubits = circ.num_qubits

    if noise_model == 'thermal':
        backend = thermal_relaxation_noise_ionq(num_qubits)[0]
    elif noise_model == 'basic':
        backend = AerSimulator(noise_model=basic_noise(num_qubits))
    elif noise_model == 'noiseless':
        backend = AerSimulator()
    
    #backend = AerSimulator(noise_model=noise_model)
    transpiled_circuit = transpile(circ, optimization_level=0, backend=backend)

    results = backend.run(transpiled_circuit, optimization_level=0, shots=1000).result()
    counts = results.get_counts(transpiled_circuit)

    total_shots = sum(counts.values())
    probabilities = {state: count / total_shots for state, count in counts.items()}

    expectation_value = probabilities['0'*num_qubits]+probabilities['1'*num_qubits]
    return expectation_value

In [42]:
# Adjusting Ella's mapping functionality:

noise_scaling_map = {
    "global": fold_global,
    "local_random": fold_gates_at_random,
    "local_all": fold_all,
    "layer": get_layer_folding,
    "identity_scaling": insert_id_layers
}

def extrapolation_map(single_exp):
    ex_map = {
        "linear": zne.inference.LinearFactory(scale_factors=single_exp['noise_scaling_factors']),
        "richardson": zne.inference.RichardsonFactory(scale_factors=single_exp['noise_scaling_factors']),
        "polynomial": zne.inference.PolyFactory(scale_factors=single_exp['noise_scaling_factors'], order=2),
        "exponential": zne.inference.ExpFactory(scale_factors=single_exp['noise_scaling_factors']),
        "poly-exp": zne.inference.PolyExpFactory(scale_factors=single_exp['noise_scaling_factors'], order=1),
        "adaptive-exp": zne.inference.AdaExpFactory(scale_factor=single_exp['noise_scaling_factors'][1], steps=4, asymptote=None), 
    }
    return ex_map[single_exp['extrapolation']]

In [43]:
# Batch execute function to return list of expectation values:

def batch_execute(batch_dict, circuit, executor):

    # Define list of experiments
    formatted_batch = make_experiment_list(batch_dict)

    # Initialize list to append expectation values into
    exp_val_list = []

    # Iterate over each experiment
    for k in formatted_batch:
        exp_val = zne.execute_with_zne(circuit=circuit, executor=executor, factory=extrapolation_map(k), 
                                       scale_noise=noise_scaling_map[k['noise_scaling_method']])
        
        # Pull out expectation values
        exp_val_list.append(exp_val)
    
    return exp_val_list

## Testing:

In [44]:
# Making a 3-qubit test GHZ circuit:

n = 3
ghz_circ = generate_ghz_circuit(n, return_type='qiskit')

ghz_circ.x([0,1])
ghz_circ.x([0,1])
ghz_circ.y([1,2])
ghz_circ.y([1,2])

print(ghz_circ)

     ┌───┐     ┌───┐┌───┐               
q_0: ┤ H ├──■──┤ X ├┤ X ├───────────────
     └───┘┌─┴─┐└───┘├───┤┌───┐┌───┐┌───┐
q_1: ─────┤ X ├──■──┤ X ├┤ X ├┤ Y ├┤ Y ├
          └───┘┌─┴─┐├───┤├───┤└───┘└───┘
q_2: ──────────┤ X ├┤ Y ├┤ Y ├──────────
               └───┘└───┘└───┘          


In [45]:
print('ideal EV: ', execute(ghz_circ, noise_model='noiseless'))
print('unmitigated noisy EV: ', execute(ghz_circ, noise_model='basic'))

ideal EV:  1.0
unmitigated noisy EV:  0.721


In [46]:
exp_results = batch_execute(zne_batch_test, ghz_circ, execute)

for k in np.arange(1,7):
    print("Experiment", k, "Mitigated Expectation Value:", exp_results[k-1])

Experiment 1 Mitigated Expectation Value: 0.7759999999999967
Experiment 2 Mitigated Expectation Value: 0.7123333333333339
Experiment 3 Mitigated Expectation Value: 0.790999999999999
Experiment 4 Mitigated Expectation Value: 0.7943333333333336
Experiment 5 Mitigated Expectation Value: 0.663999999999999
Experiment 6 Mitigated Expectation Value: 0.7863333333333338
